In [1]:
import os
import sys
import glob
import scipy
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sns.set_palette("Set2")


In [4]:
data_loc = '/home/mattolm/user_data/IndianMicrobiome/tables/'
Pdb = pd.read_csv(data_loc + 'IndianMicrobiome_basicinfo_v2.csv')
Pdb.head()


sample                                              read1  \
0    AK_SR_1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
1    AK_SR_2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
2  AK_SR_2.2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
3    AK_SR_3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   
4    AK_SR_4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...   

                                               read2            library_key  \
0  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A2_AK-SR-1   
1  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A4_AK-SR-2   
2  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...  LibPlate-D2_AK-SR-2-2   
3  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A6_AK-SR-3   
4  s3://czbiohub-microbiome/Sonnenburg_Lab/Indian...    LibPlate-A8_AK-SR-4   

   tribe  DNA_concentration sample_type fecal_sample     pairs       bases  \
0  Kabui               51.5       fecal          NaN  24276525  6938547858   
1  Kabui               28.7       fecal          NaN  25189547  7192762783   
2  Kabui               92.8       fecal          NaN  11357247  3183231459   
3  Kabui              411.8       fecal          NaN   5682989  1595248519   
4  Kabui               83.3       fecal          NaN   5552061  1578724021   

   raw_pairs    raw_bases          RL  
0   40041788  11692202096  142.906529  
1   30917233   9027832036  142.772770  
2   20045323   5853234316  140.140980  
3   11410676   3331917392  140.352948  
4    6520759   1904061628  142.174593

In [5]:
Pdb['read1'].iloc[0]

's3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R1.fastq.gz'

## Make assembly commands

In [8]:
import py_sa
import py_sa.aegea_cmds

rdb = py_sa.load_running_aegea()

for i, row in Pdb.iterrows():
    data_dest = f"{'/'.join(row['read1'].split('/')[:-2])}"
    cmd, result = py_sa.aegea_cmds.make_assembly_command(row['sample'], data_dest, row['read1'], row['read2'], ram=64000, cores=16, ret_result=True)
    
    py_sa.submit_aegea_job(cmd, result, verbose=True, rdb=rdb)
    break
    

Running showq...
Showq succeeded
1 aegea jobs are currently running
aegea batch submit --queue sonnenburg__spot100 --image sonnenburglab/single_and_coassembly:latest --storage /mnt=500 --vcpus 16 --memory 64000 --timeout 86400s --command="export coreNum=16; export mem_mb=64000; export SAMPLE=AK_SR_1; export DATA_DEST=s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1; export FWD_READS=s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R1.fastq.gz; export REV_READS=s3://czbiohub-microbiome/Sonnenburg_Lab/IndianMicrobiome/AK_SR_1/03_NODUP/NODUP_HMN_UNMAPPED_TRIM_MARKED_AK_SR_1_R2.fastq.gz; export OUT_NAME=AK_SR_1; export CONTIG_NAME=AK_SR_1; source single_sample_assembly_metaspades_merged.sh" &>> AEGEA.log
